In [3]:
from pyabsa import ABSAInstruction as absa_instruction
import os
import warnings

import findfile
from pyabsa import ABSAInstruction as absa_instruction

warnings.filterwarnings("ignore")
import pandas as pd


task_name = "multitask"
experiment_name = "instruction"

model_checkpoint = "kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined"


print("Experiment Name: ", experiment_name)
model_out_path = "ate_model_1"
model_out_path = os.path.join(
    model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}"
)
print("Model output path: ", model_out_path)

id_train_file_path = "./laptop14"
id_test_file_path = "./laptop14"

id_tr_df = absa_instruction.data_utils.read_json(id_train_file_path, "train")
id_te_df = absa_instruction.data_utils.read_json(id_test_file_path, "test")

id_tr_df = pd.DataFrame(id_tr_df)
id_te_df = pd.DataFrame(id_te_df)

loader = absa_instruction.data_utils.InstructDatasetLoader(id_tr_df, id_te_df)

if loader.train_df_id is not None:
    loader.train_df_id = loader.prepare_instruction_dataloader(loader.train_df_id)
if loader.test_df_id is not None:
    loader.test_df_id = loader.prepare_instruction_dataloader(loader.test_df_id)
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.prepare_instruction_dataloader(loader.train_df_ood)
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.prepare_instruction_dataloader(loader.test_df_ood)

# Create T5 utils object
t5_exp = absa_instruction.model.T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

# Training arguments
training_args = {
    "output_dir": model_out_path,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 5,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "load_best_model_at_end": True,
    "push_to_hub": False,
    "eval_accumulation_steps": 1,
    "predict_with_generate": True,
    "use_mps_device": False,
    "fp16": False,
}

# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

c:\Users\yingq\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


Experiment Name:  instruction
Model output path:  ate_model_1\multitask\kevinscariaate_tk-instruct-base-def-pos-neg-neut-combined-instruction
./laptop14\laptop_quad_train.tsv.jsonl
./laptop14\laptop_quad_test.tsv.jsonl


In [23]:
import pandas as pd
from pyabsa import ABSAInstruction
model = ABSAInstruction.ABSAGenerator("model")
def split_text_into_chunks(text, max_length=512):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ''

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 2 <= max_length:  # Adding 2 for the period and space
            if current_chunk:
                current_chunk += '. '  # Add period and space if it's not the first sentence in the chunk
            current_chunk += sentence
        else:
            chunks.append(current_chunk)
            current_chunk = sentence

    if current_chunk:  # Add the last chunk
        chunks.append(current_chunk)

    return chunks


df = pd.read_csv("laptop_found_intent.csv")
text = df['text'].tolist()
aspects = []
count = 0

for i in text:
    if count %100 == 0:
        f = open('x.log','a')
        f.write(str(count)+'\n')
        f.close()
    count += 1
    aspect_extracted = set()
    point = 0
    chunks = split_text_into_chunks(i)
    for chunk in chunks:
        aspect = model.predict(chunk)
        for k in aspect['Quadruples']:
            category = k['category']
            aspect_extracted.add(category)
    aspects.append(aspect_extracted)
df['aspects'] = aspects
df.to_csv('comment_with_aspect_list.csv', index=False)

c:\Users\yingq\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version 0.9.10.post1 of metric_visualizer is outdated. Version 0.9.13.post1 was released 4 days ago.
No CUDA GPU found in your device
[2024-05-06 20:03:53] (2.4.1.post1) PyABSA(2.4.1.post1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






c:\Users\yingq\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",
c:\Users\yingq\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


{'text': '**Total budget (in local currency) and country of purchase. Please do not use USD unless purchasing in the US:**  \n\\~$1,200 USD\n\n**Are you open to refurbs/used?**  \nYes if the quality isn’t affected too much\n\n**How would you prioritize form factor (ultrabook, 2-in-1, etc.), build quality, performance, and battery life?**  \nForm factor, performance, build quality, battery life\n\n**How important is weight and thinness to you?**  \nPretty important', 'Quadruples': [{'aspect': 'build quality', 'polarity': 'positive', 'opinion': 'NULL', 'category': 'LAPTOP#QUALITY'}, {'aspect': 'build quality', 'polarity': 'positive', 'opinion': 'NULL', 'category': 'LAPTOP#QUALITY'}, {'aspect': 'battery life', 'polarity': 'positive', 'opinion': 'NULL', 'category': 'LAPTOP#QUALITY'}]}
{'text': 'I will carry this for hours a day as a small adult and need it to not be too bulky for note taking\n\n**Do you have a preferred screen size? If indifferent, put N/A.**  \nPreferably 15” or smaller\n

KeyboardInterrupt: 